Automobile Alliance, a large automobile manufacturing company, organizes the vehicles it manufactures into three families: a family of trucks, a family of small cars, and a family of midsized and luxury cars. One plant outside Detroit, MI, assembles two models from the family of midsized and luxury cars. The first model, the Family Thrillseeker, is a four-door sedan with vinyl seats, plastic interior, standard features, and excellent gas mileage. It is marketed as a smart buy for middle-class families with tight budgets, and each Family Thrillseeker sold generates a modest profit of \$3,600 for the company. The second model, the Classy Cruiser, is a two-door luxury sedan with leather seats, wooden interior, custom features, and navigational capabilities. It is marketed as a privilege of affluence for upper-middle-class families, and each Classy Cruiser sold generates a healthy profit of \$5,400 for the company.

Rachel Rosencrantz, the manager of the assembly plant, is currently deciding the production schedule for the next month. Specifically, she must decide how many Family Thrillseekers and how many Classy Cruisers to assemble in the plant to maximize profit for the company. She knows that the plant possesses a capacity of 48,000 labor-hours during the month. She also knows that it takes 6 labor-hours to assemble one Family Thrillseeker and 10.5 labor-hours to assemble one Classy Cruiser.

Because the plant is simply an assembly plant, the parts required to assemble the two models are not produced at the plant. They are instead shipped from other plants around the Michigan area to the assembly plant. For example, tires, steering wheels, windows, seats, and doors all arrive from various supplier plants. For the next month, Rachel knows that she will be able to obtain only 20,000 doors (10,000 left-hand doors and 10,000 right-hand doors) from the door supplier. A recent labor strike forced the shutdown of that particular supplier plant for several days, and that plant will not be able to meet its production schedule for the next month. Both the Family Thrillseeker and the Classy Cruiser use the same door part.

In addition, a recent company forecast of the monthly demands for different automobile models suggests that the demand for the Classy Cruiser is limited to 3,500 cars. There is no limit on the demand for the Family Thrillseeker within the capacity limits of the assembly plant.

a) Formulate and solve a linear programming problem to determine the number of Family Thrillseekers and the number of Classy Cruisers that should be assembled.

In [1]:
from pyomo.environ import *
model = AbstractModel()

In [2]:
model.cars = Set()

In [3]:
model.profit = Param(model.cars)

In [4]:
model.work_hours_per_unit = Param(model.cars)
model.cars_demand = Param(model.cars)

In [5]:
model.hours_available = Param()
model.door_pairs_available = Param()

In [6]:
model.quotas = Var(model.cars, within=NonNegativeIntegers)

In [7]:
model.total_profit = Objective(rule=lambda model: summation(model.profit, model.quotas), sense=maximize)

In [8]:
model.doors_constraint = Constraint(rule=lambda model: sum(model.quotas[car] for car in model.cars) <= model.door_pairs_available)

In [9]:
model.hours_constraint = Constraint(rule=lambda model: sum(model.work_hours_per_unit[car] * 
                                                           model.quotas[car] for car in model.cars) <= model.hours_available)

In [10]:
model.demand_constraint = Constraint(model.cars, rule=lambda model, car: model.quotas[car] <= model.cars_demand[car])

In [11]:
data = { None : { 'cars' : { None : ('Family Thrillseeker', 'Classy Cruiser') }, 
                  'profit' : { 'Family Thrillseeker' : 3600, 'Classy Cruiser' : 5400 },
                  'work_hours_per_unit' : { 'Family Thrillseeker' : 6, 'Classy Cruiser' : 10.5 },
                  'cars_demand' : { 'Family Thrillseeker' : float('inf'), 'Classy Cruiser' : 3500},
                  'hours_available' : { None: 48000 },
                  'door_pairs_available' : { None: 10000 }
                }
        }

In [12]:
instance = model.create_instance(data)
instance.pprint()

1 Set Declarations
    cars : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Classy Cruiser', 'Family Thrillseeker']

5 Param Declarations
    cars_demand : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  3500
        Family Thrillseeker :   inf
    door_pairs_available : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None : 10000
    hours_available : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None : 48000
    profit : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  5400
        Family Thrillseeker :  3600
    work_hours_per_unit : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  10.5
        Family Thrillseeker :     6

1 Var Declarations


In [13]:
opt = SolverFactory('glpk')

In [14]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 28800000.0
  Upper bound: 28800000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.017337322235107422
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    total_profit:
      Value: 28800000
  Variable:
    quotas[Family Thrillseeker]:
      Value: 8000
  Constraint: No values



Before she makes her final production decisions, Rachel plans to explore the following questions independently except where otherwise indicated.

b) The marketing department knows that it can pursue a targeted $500,000 advertising campaign that will raise the demand for the Classy Cruiser next month by 20 percent. Should the campaign be undertaken?

No.

c) Rachel knows that she can increase next month’s plant capacity by using overtime labor. She can increase the plant’s labor-hour capacity by 25 percent. With the new assembly plant capacity, how many Family Thrillseekers and how many Classy Cruisers should be assembled?



In [19]:
new_capacity = 48000 * 1.25
new_capacity

60000.0

In [21]:
data = { None : { 'cars' : { None : ('Family Thrillseeker', 'Classy Cruiser') }, 
                  'profit' : { 'Family Thrillseeker' : 3600, 'Classy Cruiser' : 5400 },
                  'work_hours_per_unit' : { 'Family Thrillseeker' : 6, 'Classy Cruiser' : 10.5 },
                  'cars_demand' : { 'Family Thrillseeker' : float('inf'), 'Classy Cruiser' : 3500},
                  'hours_available' : { None: new_capacity },
                  'door_pairs_available' : { None: 10000 }
                }
        }

In [22]:
instance = model.create_instance(data)
instance.pprint()

1 Set Declarations
    cars : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Classy Cruiser', 'Family Thrillseeker']

5 Param Declarations
    cars_demand : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  3500
        Family Thrillseeker :   inf
    door_pairs_available : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None : 10000
    hours_available : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None : 60000.0
    profit : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  5400
        Family Thrillseeker :  3600
    work_hours_per_unit : Size=2, Index=cars, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             Classy Cruiser :  10.5
        Family Thrillseeker :     6

1 Var Declaration

In [23]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 36000000.0
  Upper bound: 36000000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.016556739807128906
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    total_profit:
      Value: 36000000
  Variable:
    quotas[Family Thrillseeker]:
      Value: 10000
  Constraint: No values



d) Rachel knows that overtime labor does not come without an extra cost. What is the maximum amount she should be willing to  pay for all overtime labor beyond the cost of this labor at regular time rates? Express your answer as a lump sum.

e) Rachel explores the option of using both the targeted advertising campaign and the overtime labor-hours. The advertising campaign raises the demand for the Classy Cruiser by 20 percent, and the overtime labor increases the plant’s labor-hour capacity by 25 percent. How many Family Thrillseekers and how many Classy Cruisers should be assembled using the advertising campaign and overtime labor-hours if the profit from each Classy Cruiser sold continues to be 50 percent more than for each Family Thrillseeker sold?

f) Knowing that the advertising campaign costs \$500,000 and the maximum usage of overtime labor-hours costs \$1,600,000 beyond regular time rates, is the solution found in part e) a wise decision compared to the solution found in part a)?

g) Automobile Alliance has determined that dealerships are actually heavily discounting the price of the Family Thrillseekers to move them off the lot. Because of a profit-sharing agreement with its dealers, the company is therefore not making a profit of \$3,600 on the Family Thrillseeker but is instead making a profit of \$2,800. Determine the number of Family Thrillseekers and the number of Classy Cruisers that should be assembled given this new discounted price.

h) The company has discovered quality problems with the Family Thrillseeker by randomly testing Thrillseekers at the end of the assembly line. Inspectors have discovered that in over 60 percent of the cases, two of the four doors on a Thrillseeker do not seal properly. Because the percentage of defective Thrillseekers determined by the random testing is so high, the floor supervisor has decided to perform quality control tests on every Thrillseeker at the end of the line. Because of the added tests, the time it takes to assemble one Family Thrillseeker has increased from 6 to 7.5 hours. Determine the number of units of each model that should be assembled given the new assembly time for the Family Thrillseeker.

i) The board of directors of Automobile Alliance wishes to capture a larger share of the luxury sedan market and therefore would like to meet the full demand for Classy Cruisers. They ask Rachel to determine by how much the profit of her assembly plant would decrease as compared to the profit found in part a). They then ask her to meet the full demand for Classy Cruisers if the decrease in profit is not more than $2,000,000.

j) Rachel now makes her final decision by combining all the new considerations described in parts f), g), and h). What are her final decisions on whether to undertake the advertising campaign, whether to use overtime labor, the number of Family Thrillseekers to assemble, and the number of Classy Cruisers to assemble?